<a href="https://colab.research.google.com/github/HILALOZTEMEL/BernoulliRBM-Chi_Scores-AutoEncoder-with-UNSW_NB15-dataSet/blob/main/BernoulliRBM_Chi_Scores_AutoEncoder_with_UNSW_NB15_dataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.neural_network import BernoulliRBM
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn import linear_model


#kimlik doğrulama işlemi :drive in içinde çalışma ortamı sağlarız serverlarda
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive


# Loading data from train.csv file
test_df = pd.read_csv("/gdrive/My Drive/İstanbul Gedik Üniversitesi/DataMining/archive/UNSW_NB15_testing-set.csv")
train_df = pd.read_csv("/gdrive/My Drive/İstanbul Gedik Üniversitesi/DataMining/archive/UNSW_NB15_training-set.csv")



train_df.shape, test_df.shape

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


((82332, 45), (175341, 45))

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def one_hot_encode_fit_col(data, col):
    values = data[col].unique()
    for index, v_col in enumerate(values):
        values[index] = col+"_"+v_col
        data[col+"_"+v_col] = np.zeros(len(data))
    enc = OneHotEncoder(handle_unknown='ignore')
    data[values] = enc.fit_transform(data[col].values.reshape(-1,1)).toarray()
    data = data.drop([col],axis=1)
    return data, enc, values

def one_hot_encode_transform_col(data, col, enc, col_values):
    for v_col in col_values:
        data[v_col] = np.zeros(len(data))
    data[col_values] = enc.transform(data[col].values.reshape(-1,1)).toarray()
    data = data.drop([col],axis=1)
    return data

def label_encoder_fit_col(data, col):
    values = ["out"]
    values.extend(data[col].unique())
    dict_values = {k: v for v, k in enumerate(values)}
    print(dict_values)
    data[col] = data[col].apply(lambda x: dict_values[x])
    data[col] = data[col].astype("int32")
    return data, dict_values

def label_encoder_transform_col(data, col, dict_values):
    data[col] = data[col].apply(lambda x: dict_values[x] if x in dict_values else 0)
    data[col] = data[col].astype("int32")
    return data       

In [ ]:
X_train.shape,X_test.shape , y_train.shape ,y_test.shape

((82332, 44), (175341, 44), (82332,), (175341,))

In [ ]:
train_df, dict_values = label_encoder_fit_col(train_df, "proto")
test_df = label_encoder_transform_col(test_df, "proto", dict_values)

{'out': 0, 'udp': 1, 'arp': 2, 'tcp': 3, 'igmp': 4, 'ospf': 5, 'sctp': 6, 'gre': 7, 'ggp': 8, 'ip': 9, 'ipnip': 10, 'st2': 11, 'argus': 12, 'chaos': 13, 'egp': 14, 'emcon': 15, 'nvp': 16, 'pup': 17, 'xnet': 18, 'mux': 19, 'dcn': 20, 'hmp': 21, 'prm': 22, 'trunk-1': 23, 'trunk-2': 24, 'xns-idp': 25, 'leaf-1': 26, 'leaf-2': 27, 'irtp': 28, 'rdp': 29, 'netblt': 30, 'mfe-nsp': 31, 'merit-inp': 32, '3pc': 33, 'idpr': 34, 'ddp': 35, 'idpr-cmtp': 36, 'tp++': 37, 'ipv6': 38, 'sdrp': 39, 'ipv6-frag': 40, 'ipv6-route': 41, 'idrp': 42, 'mhrp': 43, 'i-nlsp': 44, 'rvd': 45, 'mobile': 46, 'narp': 47, 'skip': 48, 'tlsp': 49, 'ipv6-no': 50, 'any': 51, 'ipv6-opts': 52, 'cftp': 53, 'sat-expak': 54, 'ippc': 55, 'kryptolan': 56, 'sat-mon': 57, 'cpnx': 58, 'wsn': 59, 'pvp': 60, 'br-sat-mon': 61, 'sun-nd': 62, 'wb-mon': 63, 'vmtp': 64, 'ttp': 65, 'vines': 66, 'nsfnet-igp': 67, 'dgp': 68, 'eigrp': 69, 'tcf': 70, 'sprite-rpc': 71, 'larp': 72, 'mtp': 73, 'ax.25': 74, 'ipip': 75, 'aes-sp3-d': 76, 'micp': 77, 'e

In [ ]:
train_df, dict_values = label_encoder_fit_col(train_df, "service")
test_df = label_encoder_transform_col(test_df, "service", dict_values)

{'out': 0, '-': 1, 'http': 2, 'ftp': 3, 'ftp-data': 4, 'smtp': 5, 'pop3': 6, 'dns': 7, 'snmp': 8, 'ssl': 9, 'dhcp': 10, 'irc': 11, 'radius': 12, 'ssh': 13}


In [ ]:
train_df, dict_values = label_encoder_fit_col(train_df, "state")
test_df = label_encoder_transform_col(test_df, "state", dict_values)

{'out': 0, 'INT': 1, 'FIN': 2, 'REQ': 3, 'ACC': 4, 'CON': 5, 'RST': 6, 'CLO': 7}


In [ ]:
train_df = train_df.sample(frac=1).reset_index(drop=True)

X_train = train_df.drop([ "attack_cat"], axis=1)
y_train = train_df["attack_cat"]
#y_train = train_df["label"]

X_test = test_df.drop([ "attack_cat"], axis=1)
y_test = test_df["attack_cat"]
#y_test = test_df["label"]

In [ ]:
rbm = BernoulliRBM(n_components=40, learning_rate=0.06, n_iter=100, verbose=1, random_state=101)
rbm.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BernoulliRBM was fitted with feature names
  "X does not have valid feature names, but"


[BernoulliRBM] Iteration 1, pseudo-likelihood = -46814183601701.42, time = 3.22s


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BernoulliRBM was fitted with feature names
  "X does not have valid feature names, but"


[BernoulliRBM] Iteration 2, pseudo-likelihood = -93630945389856.62, time = 5.78s


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BernoulliRBM was fitted with feature names
  "X does not have valid feature names, but"


...

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BernoulliRBM was fitted with feature names
  "X does not have valid feature names, but"


[BernoulliRBM] Iteration 99, pseudo-likelihood = -4634856279540930.00, time = 2.62s


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BernoulliRBM was fitted with feature names
  "X does not have valid feature names, but"


[BernoulliRBM] Iteration 100, pseudo-likelihood = -4681672868604769.00, time = 2.62s


BernoulliRBM(learning_rate=0.06, n_components=100, n_iter=100, random_state=101,
             verbose=1)

In [ ]:
X_train.shape,X_test.shape , y_train.shape ,y_test.shape

((82332, 44), (175341, 44), (82332,), (175341,))

In [ ]:
from sklearn.feature_selection import chi2

chi_scores = chi2(X_train,y_train)
chi_scores

(array([2.18009840e+08, 3.50042252e+04, 1.80119062e+06, 1.31063532e+05,
        1.54633218e+04, 5.86284040e+05, 6.02191819e+05, 1.25021854e+09,
        5.55776579e+08, 4.58568892e+09, 1.56440663e+06, 2.77456303e+06,
        1.07128669e+12, 5.89101358e+10, 3.53504802e+05, 2.36830084e+05,
        6.16357058e+07, 7.90914538e+06, 5.60025285e+08, 2.09459706e+07,
        3.65313466e+06, 2.89871209e+13, 2.88174612e+13, 3.43354646e+06,
        1.51167524e+03, 8.30788725e+02, 7.06783322e+02, 1.45778492e+06,
        4.45606276e+06, 4.17625123e+03, 8.21097864e+07, 4.87665663e+05,
        9.68865070e+03, 4.06420393e+05, 5.41306308e+05, 4.06529045e+05,
        5.99759569e+05, 7.48559272e+02, 7.43116154e+02, 4.83326423e+03,
        3.28823097e+05, 5.31397312e+05, 1.12227330e+03, 3.70000000e+04]),
 array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+

In [ ]:
X_train.shape,X_test.shape , y_train.shape ,y_test.shape

((82332, 44), (175341, 44), (82332,), (175341,))

In [ ]:
#Importing the Libraries
from sklearn.datasets import make_classification
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model


In [ ]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.



# scale data
t = MinMaxScaler()
t.fit(X_train)
X_train = t.transform(X_train)
X_test = t.transform(X_test)

# AutoEncoder Model Preparation
n_inputs = X_train.shape[1]
# define encoder
input_data_shape= Input(shape=(n_inputs,))
# encoder level 1
encoder= Dense(n_inputs*2)(input_data_shape)
encoder = BatchNormalization()(encoder)
encoder= LeakyReLU()(encoder)
# encoder level 2
encoder= Dense(n_inputs)(encoder)
encoder= BatchNormalization()(encoder)
encoder= LeakyReLU()(encoder)
# bottleneck
n_bottleneck = round(float(n_inputs) / 2.0)
bottleneck = Dense(n_bottleneck)(encoder)
# define decoder, level 1
decoder = Dense(n_inputs)(bottleneck)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU()(decoder)
# decoder level 2
decoder = Dense(n_inputs*2)(decoder)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU()(decoder)


In [ ]:
# output layer
output = Dense(n_inputs, activation='linear')(decoder)
# define autoencoder model
model = Model(inputs=input_data_shape, outputs=output)
# compile autoencoder model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# fit the autoencoder model to reconstruct input
history = model.fit(X_train, X_train,shuffle=True, epochs=5, batch_size=200, validation_data=(X_test, X_test))


Epoch 1/5
412/412 [==============================] - 81s 10ms/step - loss: 5.4840e-11 - accuracy: 1.2146e-04 - val_loss: 3.0705e-06 - val_accuracy: 0.0000e+00
Epoch 2/5
412/412 [==============================] - 5s 12ms/step - loss: 5.4443e-11 - accuracy: 1.2146e-04 - val_loss: 2.2295e-06 - val_accuracy: 0.0000e+00
Epoch 3/5
412/412 [==============================] - 4s 9ms/step - loss: 5.4237e-11 - accuracy: 1.2146e-04 - val_loss: 3.4288e-06 - val_accuracy: 0.0000e+00
Epoch 4/5
412/412 [==============================] - 4s 9ms/step - loss: 5.4092e-11 - accuracy: 1.2146e-04 - val_loss: 3.5005e-06 - val_accuracy: 2.8516e-05
Epoch 5/5
412/412 [==============================] - 4s 9ms/step - loss: 5.3978e-11 - accuracy: 1.2146e-04 - val_loss: 3.7097e-06 - val_accuracy: 0.0000e+00


In [ ]:
#Using Logistic Regression Algorithm to the Training Set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [ ]:
# Returns a NumPy Array
# Predict for One Observation (image)
classifier.predict(X_test[0].reshape(1,-1))

array(['Normal'], dtype=object)

In [ ]:
X_test.shape

(175341, 44)

In [ ]:
classifier.predict(X_test[0:10])

array(['Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal'], dtype=object)

In [ ]:
predictions = classifier.predict(X_test)

In [ ]:
# Use score method to get accuracy of model
score = classifier.score(X_test, y_test)
print(score)

0.3193776697977085
